## Putting it all together! ##

Now that we have a model and data it’s time to train, validate and test our model by optimizing it’s parameters on our data. Training a model is an iterative process; in each iteration (called an epoch) the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and optimizes these parameters using gradient descent. 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_dir = '/content/drive/MyDrive/Work/PHD/courses-AML/data'

training_data = datasets.FashionMNIST(
    root=root_dir,
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root=root_dir,
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. 
# Every nn.Module subclass implements the operations on input data in the forward method.

class NeuralNetwork(nn.Module):
    def __init__(self, input_width, input_height, label_dim):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # Used to convert the 2D input into 1D (28x28 to 784)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_width*input_height, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, label_dim),
            nn.ReLU()
        )
        self.softmax = nn.Softmax(dim=1)
        self.softmax_result = 0

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        self.softmax_result = self.softmax(logits)
        return logits

In [7]:
model = NeuralNetwork(28, 28, 10).to(device)

### Hyperparameters ### 
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates (read more about hyperparameter tuning)

We define the following hyperparameters for training:
- __Number of Epochs__ - the number times to iterate over the dataset
- __Batch Size__ - the number of data samples seen by the model in each epoch
- __Learning Rate__ - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [8]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


#### Important to know about different loss functions ####
Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss` (Mean Square Error) for regression tasks, and `nn.NLLLoss` (Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model’s output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.

Check this for more info on the loss functions that exist within Pytorch:
https://pytorch.org/docs/stable/nn.html#loss-functions

In [9]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer ##
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data. (https://pytorch.org/docs/stable/optim.html)

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(modelmodel_cnn.parameters(), lr=learning_rate)

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
- The Train Loop - iterate over the training dataset and try to converge to optimal parameters.
- The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

Inside the training loop, optimization happens in three steps:

- Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to loss.backwards(). PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        # X.shape = [64, 28, 28] [batch_size, width, height]
        # y.shape = [64] [batch_size]
        X, y = X.to(device), y.to(device) # everything in the same device
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() #Reset the gradient
        loss.backward() # Calculate the gradient
        optimizer.step() # Correct the weights

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302416  [    0/60000]
loss: 2.287087  [ 6400/60000]
loss: 2.275398  [12800/60000]
loss: 2.268215  [19200/60000]
loss: 2.242334  [25600/60000]
loss: 2.258739  [32000/60000]
loss: 2.231540  [38400/60000]
loss: 2.229340  [44800/60000]
loss: 2.222476  [51200/60000]
loss: 2.205781  [57600/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 0.034448 

Epoch 2
-------------------------------
loss: 2.204906  [    0/60000]
loss: 2.177718  [ 6400/60000]
loss: 2.152411  [12800/60000]
loss: 2.158080  [19200/60000]
loss: 2.090591  [25600/60000]
loss: 2.145612  [32000/60000]
loss: 2.085778  [38400/60000]
loss: 2.081042  [44800/60000]
loss: 2.082137  [51200/60000]
loss: 2.051567  [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 0.031843 

Epoch 3
-------------------------------
loss: 2.049833  [    0/60000]
loss: 1.994337  [ 6400/60000]
loss: 1.948065  [12800/60000]
loss: 1.976213  [19200/60000]
loss: 1.846159  [25600/60000]
loss: 1.969099  [32000/600

### Now you have trained a Neural Network to classify the Fashion MNist dataset ###

### What should you do next?? ###

- Experiment with other functionalities for Tensors and nn.Modules.
- Play with the model's parameters --> Change sizes, change activation functions, change optimizer, etc. Try different things to get a grasp of how they behave/react
- Try using the model to predict unseen clothes (Remember to deactivate the gradient in the model, and to transform the data to what the model expects). Upload your own images to see how it perform.
- Try using another dataset with all you have learned thus far (might need to read a bit more on dataset and dataloaders: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html and the link in the specific notebook) to classify something else. It does not need to be images, it can be whatever you want; try using basic tabular data.
    - Check also the other datasets they have already available in Pytorch (Image datasets: https://pytorch.org/vision/stable/datasets.html)
    - You can also try using the example dataset of the tool EDD (Evolutionary Dungeon Designer) - the labels (0 ... 11) correspond to automatically assigned clusters (https://arxiv.org/pdf/2004.01697.pdf). You got the work started in the respective notebook (dataset_dataloaders)